<a href="https://colab.research.google.com/github/ananyakbcodes/CROP_PEST_DETECTION/blob/AALIYAH/project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import joblib


In [69]:
cnn_model = load_model("/content/cnn_model.h5")   # if file is in /content
print("✅ CNN model loaded")

weather_data = joblib.load("/content/weather_model.pkl")  # also ensure file exists here
weather_model = weather_data["model"]
weather_features = weather_data["features"]
print("✅ Weather model loaded")


✅ CNN model loaded
✅ Weather model loaded


In [70]:
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (128, 128))  # MATCHING TRAINING SIZE
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    return img


In [73]:
def predict_disease_and_risk(image_path, temp, humidity, rainfall):
    # Image prediction
    img = preprocess_image(image_path)
    disease_pred = cnn_model.predict(img)
    disease_class = int(np.argmax(disease_pred))

    # --- Weather values (we fill missing model features with reasonable defaults) ---
    pressure = 1000       # <--- ADD THIS LINE
    wind_speed = 3        # <--- AND THIS ONE

    # --- Prepare weather input properly ---
    import pandas as pd
    weather_input = pd.DataFrame(
        [[temp, humidity, rainfall, pressure, wind_speed]],
        columns=["temp", "humidity", "rainfall", "pressure", "wind_speed"]
    )

    # --- Predict using the model ---
    try:
        risk_pred = weather_model.predict(weather_input)[0]
    except Exception:
        risk_pred = 0

    # --- Apply your rule for risk ---
    if humidity > 80 and 20 <= temp <= 35 and rainfall > 2:
        risk_pred = 1
        reason = "Humidity >80%, temp between 20–35°C, and rainfall >2 mm"
    else:
        risk_pred = 0
        reason = "Weather not favorable (dry or extreme conditions)"

    # --- Output ---
    print("===== FINAL PREDICTION =====")
    print("Disease Class ID:", disease_class)
    print(f"Weather Input: Temp={temp}°C, Humidity={humidity}%, Rainfall={rainfall}mm")
    print("Risk Level:", "High" if risk_pred == 1 else "Low")
    print("Reason:", reason)

In [74]:
from google.colab import files
uploaded = files.upload()


Saving OIP.webp to OIP (6).webp


In [75]:
image_path = list(uploaded.keys())[0]
image_path = "/content/" + image_path
print("Image uploaded:", image_path)

Image uploaded: /content/OIP (6).webp


In [76]:
temp = float(input("Enter current temperature (°C): "))
humidity = float(input("Enter current humidity (%): "))
rainfall = float(input("Enter rainfall (mm): "))

# Call the function with the uploaded image and entered weather data
predict_disease_and_risk(image_path, temp, humidity, rainfall)

Enter current temperature (°C): 28
Enter current humidity (%): 90
Enter rainfall (mm): 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
===== FINAL PREDICTION =====
Disease Class ID: 2
Weather Input: Temp=28.0°C, Humidity=90.0%, Rainfall=3.0mm
Risk Level: High
Reason: Humidity >80%, temp between 20–35°C, and rainfall >2 mm


In [77]:
predict_disease_and_risk(image_path, temp, humidity, rainfall)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
===== FINAL PREDICTION =====
Disease Class ID: 2
Weather Input: Temp=28.0°C, Humidity=90.0%, Rainfall=3.0mm
Risk Level: High
Reason: Humidity >80%, temp between 20–35°C, and rainfall >2 mm
